In [1]:
import os
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import pandas as pd
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt

/vol/biomedic2/zl9518/anaconda3/envs/skinclassifer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# load the model
from models.resnet_skin import network
# default settings
num_classes = 7
model = network(name = 'resnet50', num_classes = num_classes)

In [11]:
# load the model parameters
model_ckpt = './checkpoint/HAM_CE_None_0_cosine/ckpt.pth.tar'
gpu = 0
model = model.cuda(gpu)
checkpoint = torch.load(model_ckpt, map_location = 'cuda:' + str(gpu))
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [12]:
# prepare the test data.
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
ResizeTest = transforms.Resize(256)

transform_val = transforms.Compose([
        ResizeTest,
        transforms.ToTensor(),
        normalize,
    ])

data_folder = './skinlesiondatasets/SkinLesionDatasets/'
train_dataset = datasets.ImageFolder(root=os.path.join(data_folder, 'HAMtrain'), transform=transform_val)
val_dataset = datasets.ImageFolder(root=os.path.join(data_folder, 'HAMtest'), transform=transform_val)

val_dataset.class_to_idx = train_dataset.class_to_idx

val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True)

In [13]:
cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

In [14]:
cifarresultsdir = './skinresults/'
model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_val.csv'), index=False)

100%|████████████████████████| 3005/3005 [05:57<00:00,  8.41it/s]


In [15]:
# on training set.
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
ResizeTest = transforms.Resize(256)

transform_val = transforms.Compose([
        ResizeTest,
        transforms.ToTensor(),
        normalize,
    ])

data_folder = './skinlesiondatasets/SkinLesionDatasets/'
train_dataset = datasets.ImageFolder(root=os.path.join(data_folder, 'HAMtrain'), transform=transform_val)
val_dataset = datasets.ImageFolder(root=os.path.join(data_folder, 'HAMtest'), transform=transform_val)

val_dataset.class_to_idx = train_dataset.class_to_idx

val_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True)

In [16]:
cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

In [17]:
cifarresultsdir = './skinresults/'
model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_train.csv'), index=False)

100%|████████████████████████| 7010/7010 [07:48<00:00, 14.95it/s]


In [ ]:
# Other datasets here
datasetlist = ['BCN', 'D7P', 'MSK', 'PH2', 'SON', 'UDA', 'VIE']
model.eval()
cifarresultsdir = './skinresults/'
for datasetn in datasetlist:
    
    print('Processing dataset ' + datasetn)
    
    csvsavename = 'predictions_test_' + datasetn +  '.csv'
    
    val_dataset = datasets.ImageFolder(root=os.path.join(data_folder, datasetn), transform=transform_val)

    val_dataset.class_to_idx = train_dataset.class_to_idx
    
    val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=1, shuffle=False,
            num_workers=0, pin_memory=True)
    
    logits = []
    preds = []
    targets = []
    for i, (input, target) in enumerate(tqdm(val_loader)):
        input = input.cuda(gpu, non_blocking = True)
        logits_test = model(input)
        preds_test = F.softmax(logits_test, dim = 1)
        targets_test = F.one_hot(target, num_classes = num_classes)
        logits.append(logits_test.cpu().detach())
        preds.append(preds_test.cpu().detach())
        targets.append(targets_test)

    logits = torch.cat(logits, dim=0)
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)

    df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
    df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
    df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
    df = pd.concat([df, df_logits, df_targets], axis=1)
    df.to_csv(os.path.join(cifarresultsdir, csvsavename), index=False)
    

In [99]:
datasetlist = ['BCN', 'D7P', 'MSK', 'PH2', 'SON', 'UDA', 'VIE']
val_dataset = datasets.ImageFolder(root=os.path.join(data_folder, datasetlist[5]), transform=transform_val)
print(train_dataset.class_to_idx)
print(val_dataset.class_to_idx)

{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'vascular lesion': 6}
{'basal cell carcinoma': 0, 'dermatofibroma': 1, 'melanoma': 2, 'nevus': 3, 'pigmented benign keratosis': 4}


In [94]:
# corrupted datasets here
cifarresultsdir = './skinresults/'
data_folder = './skinlesiondatasets/SkinLeisionDatasets_C/'

datasetlist = ['brightness', 'contrast', 'defocus_blur', 'elastic_transform', 'fog', 
               'frost', 'gaussian_blur', 'gaussian_noise', 'glass_blur', 'impulse_noise', 
               'jpeg_compression', 'motion_blur', 'pixelate', 'saturate', 'shot_noise', 
               'snow', 'spatter', 'speckle_noise', 'zoom_blur']
severitylist = [1, 2, 3, 4, 5]

model.eval()
for datasetn in datasetlist:
    
    for severityindex in severitylist:
    
        print('Processing dataset ' + datasetn + ' severity ' + str(severityindex))

        csvsavename = 'predictions_test_' + datasetn + '_' + str(severityindex) + '.csv'

        val_dataset = datasets.ImageFolder(root=os.path.join(data_folder, datasetn, str(severityindex)), transform=transform_val)
        
        val_dataset.class_to_idx = train_dataset.class_to_idx

        val_loader = torch.utils.data.DataLoader(
                val_dataset, batch_size=1, shuffle=False,
                num_workers=0, pin_memory=True)

        logits = []
        preds = []
        targets = []
        for i, (input, target) in enumerate(tqdm(val_loader)):
            input = input.cuda(gpu, non_blocking = True)
            logits_test = model(input)
            preds_test = F.softmax(logits_test, dim = 1)
            targets_test = F.one_hot(target, num_classes = num_classes)
            logits.append(logits_test.cpu().detach())
            preds.append(preds_test.cpu().detach())
            targets.append(targets_test)

        logits = torch.cat(logits, dim=0)
        preds = torch.cat(preds, dim=0)
        targets = torch.cat(targets, dim=0)

        df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
        df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
        df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
        df = pd.concat([df, df_logits, df_targets], axis=1)
        df.to_csv(os.path.join(cifarresultsdir, csvsavename), index=False)
    

Processing dataset brightness severity 1


100%|███████████████| 3005/3005 [00:41<00:00, 72.97it/s]


Processing dataset brightness severity 2


100%|███████████████| 3005/3005 [02:24<00:00, 20.81it/s]


Processing dataset brightness severity 3


...

100%|███████████████| 3005/3005 [01:38<00:00, 30.54it/s]


Processing dataset zoom_blur severity 4


100%|███████████████| 3005/3005 [01:38<00:00, 30.52it/s]


Processing dataset zoom_blur severity 5


100%|███████████████| 3005/3005 [01:42<00:00, 29.22it/s]
